In [10]:
import pandas as pd
import os

In [11]:
DATA_DIR = '../../data/'

committee = pd.read_csv(DATA_DIR+'inputs/committee.csv')
house_committees = pd.read_csv(DATA_DIR+'inputs/house_committees.csv')
senate_committees = pd.read_csv(DATA_DIR+'inputs/senators_committees.csv')
transactions = pd.read_csv(DATA_DIR+'inputs/transactions.csv')

In [24]:
# merge house and senate committees
committees = pd.concat([house_committees, senate_committees])
# take Party.1 and take first letter and put in Party column then drop Party.1
committees['Party'] = committees['Party.1'].str[0]
# if senator is not null set chamber to senate else house
committees['chamber'] = committees['senator'].apply(lambda x: 'senate' if pd.notnull(x) else 'house')

committees = committees.drop(columns=['Party.1', 'senator', 'Office Room', 'Phone'])
committees

,State,District,Party,first_name,last_name,full_name,committee_1,committee_2,committee_3,committee_4,committee_5,committee_6,committee_7,committee_8,chamber
0,Alabama,1st,NaN,Carl,Jerry,Carl Jerry,Appropriations,Natural Resources,NaN,NaN,NaN,NaN,NaN,NaN,house
1,Alabama,2nd,NaN,Moore,Barry,Moore Barry,Agriculture,Judiciary,NaN,NaN,NaN,NaN,NaN,NaN,house
2,Alabama,3rd,NaN,Rogers,Mike,Rogers Mike,Armed Services,NaN,NaN,NaN,NaN,NaN,NaN,NaN,house
3,Alabama,4th,NaN,Aderholt,Robert,Aderholt Robert,Appropriations,NaN,NaN,NaN,NaN,NaN,NaN,NaN,house
4,Alabama,5th,NaN,Strong,Dale,Strong Dale,Armed Services,Homeland Security,"Science, Space, and Technology",NaN,NaN,NaN,NaN,NaN,house
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,Vermont,NaN,D,Peter,Welch,Peter Welch,"Committee on Agriculture, Nutrition, and Forestry","Committee on Commerce, Science, and Transporta...",Committee on Rules and Administration,Committee on the Judiciary,Joint Economic Committee,NaN,NaN,NaN,senate
96,Rhode Island,NaN,D,Sheldon,Whitehouse,Sheldon Whitehouse,Commission on Security and Cooperation in Europe,Committee on Environment and Public Works,Committee on Finance,Committee on the Budget,Committee on the Judiciary,United States Senate Caucus on International N...,NaN,NaN,senate
97,Mississippi,NaN,R,Roger,Wicker,Roger F. Wicker,Commission on Security and Cooperation in Europe,Committee on Armed Services,"Committee on Commerce, Science, and Transporta...",Committee on Environment and Public Works,Committee on Rules and Administration,Select Committee on Intelligence,NaN,NaN,senate
98,Oregon,NaN,D,Ron,Wyden,Ron Wyden,Committee on Energy and Natural Resources,Committee on Finance,Committee on the Budget,Joint Committee on Taxation,Select Committee on Intelligence,NaN,NaN,NaN,senate


In [23]:
# I want to see the merge of full_name in committees and name in transactions and then list out who match and who doesn't
# merge on name
merged = pd.merge(committees, transactions, left_on='full_name', right_on='name', how='outer')

# list out who match and who doesn't
print('Matched:')
print(len(merged[merged['full_name'].notnull() & merged['name'].notnull()]))
print('Unmatched:')
print(len(merged[merged['full_name'].isnull() & merged['name'].notnull()]))
# print out the unmatched full_names unique
print(merged[merged['full_name'].isnull() & merged['name'].notnull()]['name'].unique())
print('Missing:')
print(len(merged[merged['full_name'].notnull() & merged['name'].isnull()]))



Matched:
3217
Unmatched:
20066
['Abigail Spanberger' 'Ada Norah Henriquez' 'Adam B. Schiff'
 'Adam Kinzinger' 'Adrian Smith' 'Alan S. Lowenthal' 'Andrew Garbarino'
 'Angus S. King, Jr.' 'Ann Wagner' 'Anthony E. Gonzalez'
 'Ashley Hinson Arenholz' 'Aston Donald McEachin' 'August Lee Pfluger'
 'Austin Scott' 'Bill Flores' 'Bill Huizenga' 'Bill Pascrell' 'Billy Long'
 'Blake Moore' 'Bob Gibbs' 'Brad Sherman' 'Bradley S. Schneider'
 'Brian Higgins' 'Brian Mast' 'Carol Devine Miller' 'Carolyn B. Maloney'
 'Charles J. "Chuck" Fleischmann' 'Chellie Pingree' 'Cheri Bustos'
 'Chris Jacobs' 'Cliff Bentz' 'Cynthia Axne' 'Dan Crenshaw'
 'Dan Daniel Bishop' 'Dan Newhouse' 'Daniel Meuser' 'David B. McKinley'
 'David Cheston Rouzer' 'David E. Price' 'David Kustoff'
 'David Madison Cawthorn' 'David P. Joyce' 'David P. Roe' 'David Perdue'
 'David Trone' 'Dean Phillips' 'Debbie Dingell' 'Debbie Lesko'
 'Debbie Wasserman Schultz' 'Deborah K. Ross' 'Deborah Ross'
 'Diana Harshbarger' 'Donald S. Beyer, Jr.

In [6]:
# Normalize column names
committees = senate_committees.rename(columns=lambda x: x.strip())
committee = committee.rename(columns=lambda x: x.strip())
transactions = transactions.rename(columns=lambda x: x.strip())

# Combine house and senate committees
combined_committees = pd.concat([house_committees, senate_committees], ignore_index=True)

# Merge transactions with committees on name and check committee match
insider_transactions = pd.merge(transactions, combined_committees, left_on='name', right_on='full_name')

insider_transactions
# # Function to check sector and committee match
# def is_insider(row):
#     for i in range(1, 9):  # assuming up to 8 committees
#         if pd.notna(row[f'committee_{i}']):
#             if any(row['sector'] == committee[committee['Senate Committee'] == row[f'committee_{i}']].iloc[:, 1:].values):
#                 return True
#     return False

# # Apply the function
# insider_transactions['is_insider'] = insider_transactions.apply(is_insider, axis=1)

# df = insider_transactions[['disclosure_date', 'transaction_date', 'ticker', 'type', 'amount', 'name', 'asset_description', 'state', 'party', 'industry', 'sector', 'is_insider']]

,disclosure_date,transaction_date,ticker,type,amount,name,asset_description,state,party,industry,...,committee_2,committee_3,committee_4,senator,first_name,committee_5,committee_6,committee_7,committee_8,Party.1
0,2020-04-10,2020-03-16,FTDR,sale_full,"$1,001 - $15,000",Peter Welch,"Frontdoor, inc.",VT,Democrat,Other Consumer Services,...,"Committee on Commerce, Science, and Transporta...",Committee on Rules and Administration,Committee on the Judiciary,"Welch, Peter",Peter,Joint Economic Committee,NaN,NaN,NaN,Democratic
1,2020-04-10,2020-03-17,GIS,sale_full,"$1,001 - $15,000",Peter Welch,"General Mills, Inc.",VT,Democrat,Packaged Foods,...,"Committee on Commerce, Science, and Transporta...",Committee on Rules and Administration,Committee on the Judiciary,"Welch, Peter",Peter,Joint Economic Committee,NaN,NaN,NaN,Democratic
2,2020-04-10,2020-03-09,MDT,purchase,"$1,001 - $15,000",Peter Welch,Medtronic plc. Ordinary Shares,VT,Democrat,Biotechnology: Electromedical & Electrotherape...,...,"Committee on Commerce, Science, and Transporta...",Committee on Rules and Administration,Committee on the Judiciary,"Welch, Peter",Peter,Joint Economic Committee,NaN,NaN,NaN,Democratic
3,2020-04-10,2020-04-03,MDT,sale_full,"$1,001 - $15,000",Peter Welch,Medtronic plc. Ordinary Shares,VT,Democrat,Biotechnology: Electromedical & Electrotherape...,...,"Committee on Commerce, Science, and Transporta...",Committee on Rules and Administration,Committee on the Judiciary,"Welch, Peter",Peter,Joint Economic Committee,NaN,NaN,NaN,Democratic
4,2020-04-10,2020-03-20,PYPL,purchase,"$1,001 - $15,000",Peter Welch,"PayPal Holdings, Inc.",VT,Democrat,Business Services,...,"Committee on Commerce, Science, and Transporta...",Committee on Rules and Administration,Committee on the Judiciary,"Welch, Peter",Peter,Joint Economic Committee,NaN,NaN,NaN,Democratic
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3212,2015-01-09,2014-12-11,WASCX,purchase,"$1,001 - $15,000",Susan M. Collins,Ivy Asset Strategy C (NASDAQ),ME,Republican,NaN,...,"Committee on Health, Education, Labor, and Pen...",Select Committee on Intelligence,NaN,"Collins, Susan M.",Susan,NaN,NaN,NaN,NaN,Republican
3213,2015-01-09,2014-12-10,MDDVX,purchase,"$1,001 - $15,000",Susan M. Collins,BlackRock Equity Dividend Inv A (NASDAQ),ME,Republican,NaN,...,"Committee on Health, Education, Labor, and Pen...",Select Committee on Intelligence,NaN,"Collins, Susan M.",Susan,NaN,NaN,NaN,NaN,Republican
3214,2015-01-09,2014-12-10,MDDVX,purchase,"$1,001 - $15,000",Susan M. Collins,BlackRock Equity Dividend Inv A (NASDAQ),ME,Republican,NaN,...,"Committee on Health, Education, Labor, and Pen...",Select Committee on Intelligence,NaN,"Collins, Susan M.",Susan,NaN,NaN,NaN,NaN,Republican
3215,2015-01-09,2015-12-11,DVFAX,purchase,"$1,001 - $15,000",Susan M. Collins,Cohen &amp; Steers Dividend Value A (NASDAQ),ME,Republican,NaN,...,"Committee on Health, Education, Labor, and Pen...",Select Committee on Intelligence,NaN,"Collins, Susan M.",Susan,NaN,NaN,NaN,NaN,Republican


In [ ]:
if not os.path.exists(DATA_DIR+'outputs'):
    os.makedirs(DATA_DIR+'outputs')
df.to_csv(DATA_DIR+'outputs/trades.csv', index=False)